In [1]:
from pydantic import BaseModel, Field
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

c:\RAG\learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MovieReview(BaseModel):
    """ Class is used by LLM to get Movie Review """

    title: str = Field(max_length=500, description="Define the Movie Name")
    hero: str = Field(max_length=500, description="Provide the name of Lead Male Role")
    heroin: str = Field(max_length=500, description="Provide the name of Lead Female Role")
    year: int = Field( description= "Provide Year of Movie Release Date")
    revenue: float = Field( default="Define total world wide revenue in dollors")
    makingCost: float = Field(  default="total Making cost of Movie")
    hitOrFlot: str = Field(description=" Movie is Hit or Flop")
    ratting: float = Field(ge=0.0, le=10.0, description="get IMDB ratting")

In [3]:
wiki_tool =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
                                                        top_k_results=3,
                                                        doc_content_chars_max=2000
                                                        )
                               )

In [4]:
tavily_tool = TavilySearch(
    max_results = 5,
    topic = "general"
)

In [5]:
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai"
)
llm_travily = llm.bind_tools([ wiki_tool])

In [6]:
model = llm_travily.with_structured_output(MovieReview, include_raw = True)

In [7]:
result = model.invoke("Tell me about movie 'Dil'")

In [8]:
result

{'raw': AIMessage(content='{"title":"Dil","hero":"Aamir Khan","heroin":"Madhuri Dixit","year":1990,"revenue":200000000,"makingCost":18000000,"hitOrFlot":"Hit","ratting":7.0}', additional_kwargs={'parsed': MovieReview(title='Dil', hero='Aamir Khan', heroin='Madhuri Dixit', year=1990, revenue=200000000.0, makingCost=18000000.0, hitOrFlot='Hit', ratting=7.0), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 209, 'total_tokens': 262, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-CmnwaSlaCVmXXSBQbPQjZdNHU11rH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b1ec4-b026-7dc0-94b9-11be1820a6a3-0', usage_metadata={'input_tokens': 209